## MVO Weights
4.57606371261345,9.67698598407907,2.8816471398668,3.76280675093989,6.10005496604961,1.14708159257701,1.63996379334881,3.36411165704113,
2.00796787062891,9.13229658964017,1.37274941345458,2.38654278474067,6.25880838891221,2.14733058728827,9.76228755822564,2.01419193025687,
7.83900398874845,7.72870344184099,2.27786673057532,3.33226384976688,1.03163727917332,7.21709695950646,0.606271511043036,1.73626551968245


In [1]:
import pandas as pd
import numpy as np
import random
import math
from datetime import date,timedelta

In [2]:
def BALANCE(weights):
  #Making sure the total sum of the weights eual to 1
  weights = [w/sum(weights) for w in weights] 
  # Making sure all weights represent proportions that add up to 1
  return weights

In [3]:
def ratio(a,b,c):                       
  #function to calculate ratio i.e. "(returns-(risk_free_rate))/deviation"
  #calculating sharpe ratio
  return (a-c)/b

In [4]:
# def number_of_years(y):#calculates the number of years of the dataset
#   p=y.index[0]         #date of first row in the dataset (datetime format)
#   q=y.index[len(y)-1]  #date of last row in the dataset  (datetime format)
#   return ((q-p).days+1)/365

In [5]:
df=pd.read_csv("2021-01-15.csv",parse_dates=['Date'],index_col='Date')  #Importing Dataset
#df = df.loc["2016-01-01" :"2020-12-31" ]   #Since 2016-01-01, 5y(1234rows till 2020-12-31)
tdf=df.copy()                  #deep copy
df.reset_index(drop=True, inplace=True)
col=list(df.columns)
tdf.shape

(3, 24)

In [6]:
end=tdf.index[2].date()

In [7]:
trading_days=250#len(df)/number_of_years(tdf) 

In [8]:
returnsh=df.pct_change()                  
#Here, returnsh would mean return considered for sharpe ratio
returnsh.fillna(0,inplace=True)

In [9]:
returnsh=returnsh.iloc[1:,:]
returnsh

,ASIANPAINT,BAJFINANCE,BAJAJFINSV,BRITANNIA,DIVISLAB,GRASIM,HCLTECH,HDFCBANK,HINDALCO,HINDUNILVR,...,NESTLEIND,POWERGRID,RELIANCE,TCS,TATACONSUM,TATASTEEL,TECHM,TITAN,ULTRACEMCO,WIPRO
1,-0.014721,-0.005211,-0.005265,0.012234,0.001425,-0.012677,-0.026753,-0.001292,-0.004759,0.011044,...,0.006288,0.000488,0.011244,0.031003,-0.010316,-0.003948,-0.015940,-0.010526,-0.012105,-0.010131
2,-0.027404,0.001281,-0.012718,-0.015387,-0.021451,0.012546,-0.036878,-0.001430,-0.030796,-0.019762,...,-0.016467,-0.010239,-0.011808,-0.005337,-0.018892,-0.000637,-0.043036,-0.008524,-0.023306,-0.034775


In [10]:
returnso = returnsh.copy()  # this cell considers only NEGATIVE returns so as to calculate sortino ratio
for cols in returnso.columns.tolist():
  for i in range(1, len(df)):
    if returnso[cols][i] > 0:
      returnso[cols][i] = 0


In [11]:
returnso

,ASIANPAINT,BAJFINANCE,BAJAJFINSV,BRITANNIA,DIVISLAB,GRASIM,HCLTECH,HDFCBANK,HINDALCO,HINDUNILVR,...,NESTLEIND,POWERGRID,RELIANCE,TCS,TATACONSUM,TATASTEEL,TECHM,TITAN,ULTRACEMCO,WIPRO
1,-0.014721,-0.005211,-0.005265,0.000000,0.000000,-0.012677,-0.026753,-0.001292,-0.004759,0.000000,...,0.000000,0.000000,0.000000,0.000000,-0.010316,-0.003948,-0.015940,-0.010526,-0.012105,-0.010131
2,-0.027404,0.000000,-0.012718,-0.015387,-0.021451,0.000000,-0.036878,-0.001430,-0.030796,-0.019762,...,-0.016467,-0.010239,-0.011808,-0.005337,-0.018892,-0.000637,-0.043036,-0.008524,-0.023306,-0.034775


In [12]:
covmatsh=returnsh.cov()*trading_days     
#Annualised covariance matrix calculated wrt returnsh i.e. used to calculate sharpe ratio
covmatso = returnso.cov() * trading_days

In [13]:
covmatsh

,ASIANPAINT,BAJFINANCE,BAJAJFINSV,BRITANNIA,DIVISLAB,GRASIM,HCLTECH,HDFCBANK,HINDALCO,HINDUNILVR,...,NESTLEIND,POWERGRID,RELIANCE,TCS,TATACONSUM,TATASTEEL,TECHM,TITAN,ULTRACEMCO,WIPRO
ASIANPAINT,0.020107,-0.010293,0.011816,0.043789,0.036268,-0.039988,0.016052,0.000218,0.041277,0.048839,...,0.036075,0.017006,0.036545,0.057612,0.013597,-0.005250,0.042958,-0.003174,0.017758,0.039070
BAJFINANCE,-0.010293,0.005269,-0.006049,-0.022417,-0.018566,0.020471,-0.008218,-0.000112,-0.021131,-0.025002,...,-0.018468,-0.008706,-0.018708,-0.029493,-0.006961,0.002688,-0.021991,0.001625,-0.009091,-0.020001
BAJAJFINSV,0.011816,-0.006049,0.006943,0.025732,0.021312,-0.023499,0.009433,0.000128,0.024256,0.028700,...,0.021199,0.009993,0.021476,0.033855,0.007990,-0.003085,0.025244,-0.001865,0.010435,0.022959
BRITANNIA,0.043789,-0.022417,0.025732,0.095362,0.078983,-0.087085,0.034958,0.000476,0.089893,0.106360,...,0.078564,0.037036,0.079588,0.125466,0.029611,-0.011433,0.093552,-0.006913,0.038672,0.085087
DIVISLAB,0.036268,-0.018566,0.021312,0.078983,0.065416,-0.072127,0.028954,0.000394,0.074452,0.088091,...,0.065069,0.030674,0.065917,0.103915,0.024525,-0.009469,0.077483,-0.005726,0.032030,0.070472
GRASIM,-0.039988,0.020471,-0.023499,-0.087085,-0.072127,0.079526,-0.031924,-0.000435,-0.082090,-0.097128,...,-0.071744,-0.033821,-0.072679,-0.114576,-0.027041,0.010441,-0.085432,0.006313,-0.035316,-0.077701
HCLTECH,0.016052,-0.008218,0.009433,0.034958,0.028954,-0.031924,0.012815,0.000174,0.032953,0.038990,...,0.028800,0.013577,0.029175,0.045994,0.010855,-0.004191,0.034295,-0.002534,0.014177,0.031191
HDFCBANK,0.000218,-0.000112,0.000128,0.000476,0.000394,-0.000435,0.000174,0.000002,0.000449,0.000531,...,0.000392,0.000185,0.000397,0.000626,0.000148,-0.000057,0.000467,-0.000034,0.000193,0.000425
HINDALCO,0.041277,-0.021131,0.024256,0.089893,0.074452,-0.082090,0.032953,0.000449,0.084737,0.100260,...,0.074058,0.034911,0.075023,0.118270,0.027913,-0.010778,0.088187,-0.006516,0.036454,0.080206
HINDUNILVR,0.048839,-0.025002,0.028700,0.106360,0.088091,-0.097128,0.038990,0.000531,0.100260,0.118626,...,0.087624,0.041307,0.088766,0.139935,0.033026,-0.012752,0.104341,-0.007710,0.043132,0.094899


In [14]:
risk_free_rate = 0.0358 #initializing risk free rate that will be used in calculating both the ratios (absolute value)
#referred from url: https://www.rbi.org.in/Scripts/BS_NSDPDisplay.aspx?param=4&Id=24292
#In the above url, the 364 (1 year) day treasury bill is 3.58% , when taken absolute value => 0.0358
# (improved)

In [15]:
df
stocks=df.shape[1]
stocks

24

# Sharpe


In [16]:
global_warr_sortino=[]
global_war_sharpe=[]
sharpe_portfolio_return=[]
sharpe_portfolio_risk=[]
sharpe_portfolio_shratio=[]
sharpe_portfolio_stockWeights=[]

In [17]:

fitness=0
#for each iteration

weights=np.array(BALANCE(np.array([4.57606371261345,9.67698598407907,2.8816471398668,3.76280675093989,6.10005496604961,1.14708159257701,1.63996379334881,3.36411165704113,2.00796787062891,9.13229658964017,1.37274941345458,2.38654278474067,6.25880838891221,2.14733058728827,9.76228755822564,2.01419193025687,7.83900398874845,7.72870344184099,2.27786673057532,3.33226384976688,1.03163727917332,7.21709695950646,0.606271511043036,1.73626551968245
])))

#calculating annulaised portfolio return
returns_temp = np.sum(returnsh.mean()*weights)*trading_days 

#calculating portfolio varience wrt calculating sharpe ratio
varsh=np.dot(weights.T,np.dot(covmatsh,weights))   

#portfolio risk
volatility_temp = np.sqrt(varsh)      

#Calculating fitness value(ie sharpe ratio)
fitness = ratio(returns_temp,volatility_temp,risk_free_rate)

#Initializing the intial fitness value as the best fitness value(sharpe_pbest)


sharpe_portfolio_return.append(returns_temp)
sharpe_portfolio_risk.append(volatility_temp)
sharpe_portfolio_shratio.append(fitness)
sharpe_portfolio_stockWeights.append(weights)



In [18]:
sharpe_portfolio = {'Returns' : sharpe_portfolio_return, 'Standard Deviation' : sharpe_portfolio_risk,  'Sharpe Ratio' : sharpe_portfolio_shratio}  

for counter,symbol in enumerate(df.columns):
  sharpe_portfolio[symbol + " Weight"] = [Weight[counter] for Weight in sharpe_portfolio_stockWeights]
sharpe_pc = pd.DataFrame(sharpe_portfolio)
sharpe_optimal=sharpe_pc.iloc[sharpe_pc['Sharpe Ratio'].idxmax()]
sharpe_optimal=pd.DataFrame(sharpe_optimal)
sharpe_optimal.to_csv("sharpe_optimal.csv")

sharpe_pc

,Returns,Standard Deviation,Sharpe Ratio,ASIANPAINT Weight,BAJFINANCE Weight,BAJAJFINSV Weight,BRITANNIA Weight,DIVISLAB Weight,GRASIM Weight,HCLTECH Weight,...,NESTLEIND Weight,POWERGRID Weight,RELIANCE Weight,TCS Weight,TATACONSUM Weight,TATASTEEL Weight,TECHM Weight,TITAN Weight,ULTRACEMCO Weight,WIPRO Weight
0,-1.532218,0.156367,-10.0278,0.045761,0.09677,0.028816,0.037628,0.061001,0.011471,0.0164,...,0.097623,0.020142,0.07839,0.077287,0.022779,0.033323,0.010316,0.072171,0.006063,0.017363


In [19]:
sharpe_pc.to_csv(str(end)+'MVOsharpe_A.csv')

# Sortino

In [20]:
global_warr_sortino=[]
sortino_portfolio_return=[]
sortino_portfolio_risk=[]
sortino_portfolio_soratio=[]
sortino_portfolio_stockWeights=[]

In [21]:

fitness=0
#for each iteration


#Making sure that the total amount of pheromone equals 1 
weights=np.array(BALANCE([4.57606371261345,9.67698598407907,2.8816471398668,3.76280675093989,6.10005496604961,1.14708159257701,1.63996379334881,3.36411165704113,2.00796787062891,9.13229658964017,1.37274941345458,2.38654278474067,6.25880838891221,2.14733058728827,9.76228755822564,2.01419193025687,7.83900398874845,7.72870344184099,2.27786673057532,3.33226384976688,1.03163727917332,7.21709695950646,0.606271511043036,1.73626551968245
]))

#calculating annulaised portfolio return
returns_temp = np.sum(returnsh.mean()*weights)*trading_days 

#calculating portfolio varience wrt calculating sharpe ratio
varso=np.dot(weights.T,np.dot(covmatso,weights))   

#portfolio risk
volatility_temp = np.sqrt(varso)      

#Calculating fitness value(ie sortino ratio)
fitness = ratio(returns_temp,volatility_temp,risk_free_rate)

#Initializing the intial fitness value as the best fitness value(sortino_pbest)

sortino_portfolio_return.append(returns_temp)
sortino_portfolio_risk.append(volatility_temp)
sortino_portfolio_soratio.append(fitness)
sortino_portfolio_stockWeights.append(weights)


In [22]:
sortino_portfolio = {'Returns' : sortino_portfolio_return, 'Semi Deviation' : sortino_portfolio_risk,  'Sortino Ratio' : sortino_portfolio_soratio}  

for counter,symbol in enumerate(df.columns):
  sortino_portfolio[symbol + " Weight"] = [Weight[counter] for Weight in sortino_portfolio_stockWeights]
sortino_pc = pd.DataFrame(sortino_portfolio)
sortino_optimal=sortino_pc.iloc[sortino_pc['Sortino Ratio'].idxmax()]
sortino_optimal=pd.DataFrame(sortino_optimal)
sortino_optimal.to_csv("sortino_optimal.csv")
sortino_pc


,Returns,Semi Deviation,Sortino Ratio,ASIANPAINT Weight,BAJFINANCE Weight,BAJAJFINSV Weight,BRITANNIA Weight,DIVISLAB Weight,GRASIM Weight,HCLTECH Weight,...,NESTLEIND Weight,POWERGRID Weight,RELIANCE Weight,TCS Weight,TATACONSUM Weight,TATASTEEL Weight,TECHM Weight,TITAN Weight,ULTRACEMCO Weight,WIPRO Weight
0,-1.532218,0.094074,-16.667994,0.045761,0.09677,0.028816,0.037628,0.061001,0.011471,0.0164,...,0.097623,0.020142,0.07839,0.077287,0.022779,0.033323,0.010316,0.072171,0.006063,0.017363


In [23]:

sortino_pc.to_csv(str(end)+'MVOsortino_A.csv')